# Pytorch->Onnx->CoreML Walkthrough

Basic CNN from https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

import onnx
import onnx_coreml

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

In [3]:
trainset = torchvision.datasets.CIFAR10(root='./data',
                                       train=True,
                                       download=True,
                                       transform=transform)

# Set batch-size to 1 for ONNX Trace
trainloader = torch.utils.data.DataLoader(trainset,
                                         batch_size=1,
                                         shuffle=True,
                                         num_workers=2)

Files already downloaded and verified


In [4]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [5]:
dataiter = iter(trainloader)
images, labels = dataiter.next()

print(images[0].shape)

torch.Size([3, 32, 32])


`dataiter.next()` returns `list(tensor(imgs...), tensor(labels...))`

For ONNX export we only need the network input so it can perform a model trace.

In [6]:
dataiter.next()

[tensor([[[[ 0.0745,  0.1216,  0.1451,  ...,  0.0667,  0.0667,  0.0745],
           [ 0.0980,  0.1373,  0.1529,  ...,  0.0353,  0.0275,  0.0353],
           [ 0.1059,  0.1373,  0.1686,  ..., -0.0039, -0.0039,  0.0039],
           ...,
           [ 0.4353,  0.3961,  0.3569,  ...,  0.0118,  0.1216,  0.2549],
           [ 0.4353,  0.3882,  0.3490,  ...,  0.1059,  0.0118,  0.3098],
           [ 0.3647,  0.3490,  0.2941,  ...,  0.3020,  0.2157,  0.3490]],
 
          [[ 0.1294,  0.1529,  0.1608,  ...,  0.3333,  0.3333,  0.3412],
           [ 0.1451,  0.1608,  0.1686,  ...,  0.3020,  0.2941,  0.3020],
           [ 0.1373,  0.1529,  0.1686,  ...,  0.2706,  0.2706,  0.2784],
           ...,
           [ 0.2549,  0.2078,  0.1373,  ...,  0.1608,  0.2000,  0.3725],
           [ 0.2471,  0.1922,  0.1294,  ...,  0.1765,  0.0196,  0.3961],
           [ 0.1686,  0.1451,  0.0667,  ...,  0.3098,  0.1765,  0.3961]],
 
          [[-0.0118,  0.0196,  0.0353,  ...,  0.5608,  0.5529,  0.5608],
           [ 

## Define a simple CNN

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

## Export to ONNX Model

In [9]:
this_model_name = 'e2e-test-dev-03'
onnx_model_path = this_model_name + '.onnx'

torch.onnx.export(net, dataiter.next()[0], onnx_model_path, verbose=True)

graph(%input.1 : Float(1, 3, 32, 32),
      %conv1.weight : Float(6, 3, 5, 5),
      %conv1.bias : Float(6),
      %conv2.weight : Float(16, 6, 5, 5),
      %conv2.bias : Float(16),
      %fc1.weight : Float(120, 400),
      %fc1.bias : Float(120),
      %fc2.weight : Float(84, 120),
      %fc2.bias : Float(84),
      %fc3.weight : Float(10, 84),
      %fc3.bias : Float(10)):
  %11 : Float(1, 6, 28, 28) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[5, 5], pads=[0, 0, 0, 0], strides=[1, 1]](%input.1, %conv1.weight, %conv1.bias), scope: Net/Conv2d[conv1] # /Users/rhombe/.pyenv/versions/3.6.8/lib/python3.6/site-packages/torch/nn/modules/conv.py:340:0
  %12 : Float(1, 6, 28, 28) = onnx::Relu(%11), scope: Net # /Users/rhombe/.pyenv/versions/3.6.8/lib/python3.6/site-packages/torch/nn/functional.py:913:0
  %13 : Float(1, 6, 14, 14) = onnx::MaxPool[kernel_shape=[2, 2], pads=[0, 0, 0, 0], strides=[2, 2]](%12), scope: Net/MaxPool2d[pool] # /Users/rhombe/.pyenv/versions/3.6.8/lib/python3.

## Converting to CoreML

In [10]:
mlmodel_path = this_model_name + '.mlmodel'

onnx_model_file = open(onnx_model_path, 'rb')
model_proto = onnx.onnx_pb.ModelProto()
model_proto.ParseFromString(onnx_model_file.read())

249487

In [11]:
model_proto

ir_version: 4
producer_name: "pytorch"
producer_version: "1.2"
graph {
  node {
    input: "input.1"
    input: "conv1.weight"
    input: "conv1.bias"
    output: "11"
    op_type: "Conv"
    attribute {
      name: "dilations"
      ints: 1
      ints: 1
      type: INTS
    }
    attribute {
      name: "group"
      i: 1
      type: INT
    }
    attribute {
      name: "kernel_shape"
      ints: 5
      ints: 5
      type: INTS
    }
    attribute {
      name: "pads"
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      type: INTS
    }
    attribute {
      name: "strides"
      ints: 1
      ints: 1
      type: INTS
    }
  }
  node {
    input: "11"
    output: "12"
    op_type: "Relu"
  }
  node {
    input: "12"
    output: "13"
    op_type: "MaxPool"
    attribute {
      name: "kernel_shape"
      ints: 2
      ints: 2
      type: INTS
    }
    attribute {
      name: "pads"
      ints: 0
      ints: 0
      ints: 0
      ints: 0
      type: INTS
    }
    attribut

In [12]:
coreml_model = onnx_coreml.convert(model_proto,
                                   image_input_names=['inputImage'],
                                   image_output_names=['outputImage'],
                                   disable_coreml_rank5_mapping=True
                                  )
coreml_model.save(mlmodel_path)

_make_coreml_input_features
Collected input_features:  [('input.1', Array({1,3,32,32}))]
1/12: Converting Node Type Conv
2/12: Converting Node Type Relu
3/12: Converting Node Type MaxPool
4/12: Converting Node Type Conv
5/12: Converting Node Type Relu
6/12: Converting Node Type MaxPool
7/12: Converting Node Type Reshape
8/12: Converting Node Type Gemm
9/12: Converting Node Type Relu
10/12: Converting Node Type Gemm
11/12: Converting Node Type Relu
12/12: Converting Node Type Gemm
Translation to CoreML spec completed. Now compiling the CoreML model.
Inputs:
  input.1 [1, 3, 32, 32]
Outputs:
  23 [1, 10]


def model(input.1) :
	11 =  convolution (input.1)
	12 =  activation (11)
	13 =  pooling (12)
	14 =  convolution (13)
	15 =  activation (14)
	16 =  pooling (15)
	18 =  reshapeStatic (16)
	19 =  batchedMatmul (18)
	20 =  activation (19)
	21 =  batchedMatmul (20)
	22 =  activation (21)
	23 =  batchedMatmul (22)
 
	return 23
Model Compilation done.


In [13]:
res = net(dataiter.next()[0])

In [14]:
res.shape

torch.Size([1, 10])

In [15]:
coreml_model

input {
  name: "input.1"
  type {
    multiArrayType {
      shape: 1
      shape: 3
      shape: 32
      shape: 32
      dataType: FLOAT32
    }
  }
}
output {
  name: "23"
  type {
    multiArrayType {
      shape: 1
      shape: 10
      dataType: FLOAT32
    }
  }
}

In [17]:
import coremltools

coreml_model.predict({'input.1': dataiter.next()[0].numpy()})

{'23': array([[ 0.03159744, -0.10788564, -0.09554024,  0.04849986,  0.060378  ,
          0.10113253,  0.14130479,  0.11806699, -0.00622333,  0.04473902]],
       dtype=float32)}